In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Initial overview of the datasets

In [2]:
crime_df = pd.read_csv('./data/On_Street_Crime_In_Camden.csv')

print(crime_df.shape)

crime_df.head()

(151556, 20)


,Category,Street ID,Street Name,Context,Outcome Category,Outcome Date,Service,Location Subtype,ID,Persistent ID,Epoch,Ward Code,Ward Name,Easting,Northing,Longitude,Latitude,Spatial Accuracy,Last Uploaded,Location
0,Theft from the person,960684,On or near Conway Mews,NaN,Investigation complete; no suspect identified,01/06/2015,Police Force,NaN,41015103,8b3fa5ae8a1d7c2a018dac3ea65b48d488a72a9088c0ce...,01/05/2015,E05000129,Bloomsbury,529174.85,182002.86,-0.139515,51.522189,This is only an approximation of where the cri...,21/08/2015,"(51.522189, -0.139515)"
1,Burglary,956618,On or near Rathbone Place,NaN,Awaiting court outcome,01/05/2015,Police Force,NaN,38756146,0f2959763b802edd81a586068a3c12cea2855006aa174d...,01/01/2015,E05000129,Bloomsbury,529552.90,181517.86,-0.134247,51.517744,This is only an approximation of where the cri...,21/08/2015,"(51.517744, -0.134247)"
2,Anti-social behaviour,967781,On or near Goldhurst Terrace,NaN,NaN,NaN,Police Force,NaN,44908996,NaN,01/11/2015,E05000133,Frognal and Fitzjohns,526367.86,184618.86,-0.179014,51.546333,This is only an approximation of where the cri...,05/05/2016,"(51.546333, -0.179014)"
3,Anti-social behaviour,960974,On or near Birkenhead Street,NaN,NaN,NaN,Police Force,NaN,44521423,NaN,01/10/2015,E05000141,King's Cross,530392.86,182859.87,-0.121652,51.529611,This is only an approximation of where the cri...,05/05/2016,"(51.529611, -0.121652)"
4,Violence and sexual offences,960582,On or near Shopping Area,NaN,Under investigation,01/02/2018,Police Force,NaN,63282493,130fb7417cd86b5ad345c6c2430ace968837c93b46a08c...,01/02/2018,E05000142,Regent's Park,529585.27,182734.74,-0.133296,51.528664,This is only an approximation of where the cri...,13/04/2018,"(51.528664, -0.133296)"


In [3]:
lighting_df = pd.read_csv('./data/Camden_Street_Lighting.csv')

print(lighting_df.shape)

lighting_df.head()

(9527, 13)


,Street Name,Ward Name,Postcode,Switch Regime,Wattage,Easting,Northing,Longitude,Latitude,Spatial Accuracy,Location,Organisation URI,Last Uploaded
0,MILL LANE,FORTUNE GREEN,NW6,35/18,65.0,524803,185110,-0.201358,51.551088,Plotted By Council Officer,"(51.551088, -0.201358)",http://opendatacommunities.org/id/london-borou...,23/08/2018 09:00:09 PM
1,KING HENRY'S ROAD,SWISS COTTAGE,NW3,35/18,90.0,526991,184116,-0.170176,51.541667,Plotted By Council Officer,"(51.541667, -0.170176)",http://opendatacommunities.org/id/london-borou...,23/08/2018 09:00:15 PM
2,PRATT STREET,CAMDEN TOWN,NW1,35/18,90.0,529062,183694,-0.140483,51.537405,Plotted By Council Officer,"(51.537405, -0.140483)",http://opendatacommunities.org/id/london-borou...,23/08/2018 09:00:20 PM
3,HOLLY HILL,HAMPSTEAD TOWN,NW3,70/35,125.0,526343,185793,-0.178913,51.556883,Plotted By Council Officer,"(51.556883, -0.178913)",http://opendatacommunities.org/id/london-borou...,23/08/2018 09:00:13 PM
4,THEOBALD'S ROAD,HOLBORN AND COVENT GARDEN,WC1,35/18,90.0,530787,181882,-0.116298,51.520724,Plotted By Council Officer,"(51.520724, -0.116298)",http://opendatacommunities.org/id/london-borou...,23/08/2018 09:00:27 PM


## Analysis of street lighting dataset

In [4]:
def format_coords(street):
    '''
    Returns a list of coodinates formatted as a tuple of (longitute, latitude)
    
    Args:
        street (pandas.DataFrame): Must contain as lease two columns named 'Longiture' and 'Latitude'
    
    Returns:
        list: Contains tuples of (longitute (float), latitude (float))
        
    '''
    coords = []
    
    (street[['Longitude', 'Latitude']]
     .apply(lambda loc: coords.append((loc['Longitude'], loc['Latitude'])), axis=1))
    
    return coords

In [5]:
# Format coordinates to (long, lat)

coords_sr = (lighting_df
             .groupby(['Ward Name', 'Street Name'])
             .apply(format_coords))

coords_sr.head()

Ward Name  Street Name   
BELSIZE    ADAMSON ROAD      [(-0.171694, 51.545153000000006), (-0.171526, ...
           ANTRIM GROVE      [(-0.16188, 51.548542), (-0.162301, 51.5481349...
           ANTRIM ROAD       [(-0.162389, 51.547740999999995), (-0.161373, ...
           BELSIZE AVENUE    [(-0.16813, 51.550241), (-0.166798, 51.551074)...
           BELSIZE GROVE     [(-0.16461800000000001, 51.548244), (-0.16538,...
dtype: object

In [6]:
# Calculate total wattage of each street

wattage_sr = (lighting_df
              .groupby(['Ward Name', 'Street Name'])
              .apply(lambda street: np.sum(street['Wattage'])))

wattage_sr.head()

Ward Name  Street Name   
BELSIZE    ADAMSON ROAD       504.0
           ANTRIM GROVE       180.0
           ANTRIM ROAD        360.0
           BELSIZE AVENUE    1170.0
           BELSIZE GROVE      732.0
dtype: float64

In [7]:
# Add total_wattage column to the summary dataframe

summary_df = (pd
              .concat([coords_sr.rename('coordinates (long,lat)'), wattage_sr.rename('total wattage')], axis=1)
              .reset_index())

summary_df.head()

,Ward Name,Street Name,"coordinates (long,lat)",total wattage
0,BELSIZE,ADAMSON ROAD,"[(-0.171694, 51.545153000000006), (-0.171526, ...",504.0
1,BELSIZE,ANTRIM GROVE,"[(-0.16188, 51.548542), (-0.162301, 51.5481349...",180.0
2,BELSIZE,ANTRIM ROAD,"[(-0.162389, 51.547740999999995), (-0.161373, ...",360.0
3,BELSIZE,BELSIZE AVENUE,"[(-0.16813, 51.550241), (-0.166798, 51.551074)...",1170.0
4,BELSIZE,BELSIZE GROVE,"[(-0.16461800000000001, 51.548244), (-0.16538,...",732.0


In [8]:
# Calculate area of polygons covered by coordinates
# with transformation to accound for the roundness of the earth

import pyproj
import shapely
import shapely.ops as ops
from shapely.geometry.polygon import Polygon
from functools import partial 

def calculate_area(coords):
    if (len(coords) >= 3):
        geom = Polygon(coords)
        geom_area = ops.transform(partial(pyproj.transform,
                                          pyproj.Proj(init='EPSG:4326'),
                                          pyproj.Proj(proj='aea',
                                                      lat1=geom.bounds[1], 
                                                      lat2=geom.bounds[3])),
                                  geom)

        return geom_area.area  # in m^2

In [9]:
# Summary df with area & lit index (total_wattage / area)

summary_df['area'] = summary_df['coordinates (long,lat)'].apply(calculate_area)

summary_df['lit index'] = summary_df['total wattage'] / summary_df['area']

summary_df['Ward Name'] = summary_df['Ward Name'].str.lower()
summary_df['Street Name'] = summary_df['Street Name'].str.lower()

summary_df.head()

,Ward Name,Street Name,"coordinates (long,lat)",total wattage,area,lit index
0,belsize,adamson road,"[(-0.171694, 51.545153000000006), (-0.171526, ...",504.0,1261.545104,0.399510
1,belsize,antrim grove,"[(-0.16188, 51.548542), (-0.162301, 51.5481349...",180.0,244.861768,0.735109
2,belsize,antrim road,"[(-0.162389, 51.547740999999995), (-0.161373, ...",360.0,540.350583,0.666234
3,belsize,belsize avenue,"[(-0.16813, 51.550241), (-0.166798, 51.551074)...",1170.0,1573.712255,0.743465
4,belsize,belsize grove,"[(-0.16461800000000001, 51.548244), (-0.16538,...",732.0,639.174952,1.145226


## Are lit index & crime count correlated?

In [10]:
# Obtain the total number per category on a street

crime_count_df = (crime_df
                  .groupby(['Ward Name', 'Street Name', 'Category'])
                  .size()
                  .reset_index(name='count'))

crime_count_df['Ward Name'] = crime_count_df['Ward Name'].str.lower()
crime_count_df['Street Name'] = crime_count_df['Street Name'].str.lower()

crime_count_df.head()

,Ward Name,Street Name,Category,count
0,belsize,on or near adamson road,Anti-social behaviour,36
1,belsize,on or near adamson road,Bicycle theft,1
2,belsize,on or near adamson road,Burglary,7
3,belsize,on or near adamson road,Criminal damage and arson,6
4,belsize,on or near adamson road,Drugs,1


In [11]:
# Get a list of crime categories

crime_categories = crime_count_df['Category'].unique()

print('Crime categories')
print(crime_categories)

# Create empty columns for each category and total crime count

for category in crime_categories:
    summary_df[category] = 0
    
summary_df['total crime'] = 0

summary_df.head()

Crime categories
['Anti-social behaviour' 'Bicycle theft' 'Burglary'
 'Criminal damage and arson' 'Drugs' 'Other crime' 'Other theft'
 'Public order' 'Robbery' 'Theft from the person' 'Vehicle crime'
 'Violence and sexual offences' 'Possession of weapons' 'Shoplifting']


,Ward Name,Street Name,"coordinates (long,lat)",total wattage,area,lit index,Anti-social behaviour,Bicycle theft,Burglary,Criminal damage and arson,...,Other crime,Other theft,Public order,Robbery,Theft from the person,Vehicle crime,Violence and sexual offences,Possession of weapons,Shoplifting,total crime
0,belsize,adamson road,"[(-0.171694, 51.545153000000006), (-0.171526, ...",504.0,1261.545104,0.399510,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,belsize,antrim grove,"[(-0.16188, 51.548542), (-0.162301, 51.5481349...",180.0,244.861768,0.735109,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,belsize,antrim road,"[(-0.162389, 51.547740999999995), (-0.161373, ...",360.0,540.350583,0.666234,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,belsize,belsize avenue,"[(-0.16813, 51.550241), (-0.166798, 51.551074)...",1170.0,1573.712255,0.743465,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,belsize,belsize grove,"[(-0.16461800000000001, 51.548244), (-0.16538,...",732.0,639.174952,1.145226,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# Map counts of each crime and total count of all crime per street

def add_crime_count(entry):
    ward_filter = crime_count_df['Ward Name'] == entry['Ward Name'].iloc[0]
    street_filter = crime_count_df['Street Name'].str.lower().str.contains(entry['Street Name'].iloc[0], regex=False)
    
    matching_crime_df = crime_count_df[ward_filter & street_filter]
    
    if not matching_crime_df.empty:
        categories = matching_crime_df['Category']
        total_counts = np.sum(matching_crime_df['count'])
        entry['total crime'] = total_counts

        for category in categories:
            count = matching_crime_df[matching_crime_df['Category'] == category]['count'].iloc[0]
            entry[category] = count
        
    return entry

In [13]:
summary_df = summary_df.groupby(['Ward Name', 'Street Name']).apply(add_crime_count)

In [14]:
summary_df.head()

,Ward Name,Street Name,"coordinates (long,lat)",total wattage,area,lit index,Anti-social behaviour,Bicycle theft,Burglary,Criminal damage and arson,...,Other crime,Other theft,Public order,Robbery,Theft from the person,Vehicle crime,Violence and sexual offences,Possession of weapons,Shoplifting,total crime
0,belsize,adamson road,"[(-0.171694, 51.545153000000006), (-0.171526, ...",504.0,1261.545104,0.399510,36,1,7,6,...,2,8,4,8,7,14,17,0,0,111
1,belsize,antrim grove,"[(-0.16188, 51.548542), (-0.162301, 51.5481349...",180.0,244.861768,0.735109,88,1,15,7,...,0,1,2,1,6,4,1,0,0,127
2,belsize,antrim road,"[(-0.162389, 51.547740999999995), (-0.161373, ...",360.0,540.350583,0.666234,4,1,5,1,...,0,3,1,2,3,1,1,0,0,24
3,belsize,belsize avenue,"[(-0.16813, 51.550241), (-0.166798, 51.551074)...",1170.0,1573.712255,0.743465,15,1,9,6,...,0,8,2,2,6,9,8,0,0,67
4,belsize,belsize grove,"[(-0.16461800000000001, 51.548244), (-0.16538,...",732.0,639.174952,1.145226,33,4,12,9,...,2,4,5,1,7,16,15,0,0,108


In [15]:
# Correlation analysis of lit index vs total crime count

summary_df['lit index'].corr(summary_df['total crime'])

-0.02152955900659705

In [16]:
# Pairwise correlation for a more fine grained analysis

columns_to_drop = ['Ward Name', 
                   'Street Name', 
                   'coordinates (long,lat)',
                   'total wattage',
                   'area']

(summary_df
 .drop(columns_to_drop, axis=1)
 .corr(method='spearman')
 .style.format("{:.2}")
 .background_gradient(cmap=plt.get_cmap('coolwarm'), axis=1))

,lit index,Anti-social behaviour,Bicycle theft,Burglary,Criminal damage and arson,Drugs,Other crime,Other theft,Public order,Robbery,Theft from the person,Vehicle crime,Violence and sexual offences,Possession of weapons,Shoplifting,total crime
lit index,1.0,-0.05,-0.036,-0.096,-0.12,-0.064,-0.029,-0.045,-0.09,-0.084,-0.043,-0.14,-0.087,-0.015,0.018,-0.08
Anti-social behaviour,-0.05,1.0,0.53,0.61,0.61,0.54,0.33,0.61,0.61,0.58,0.6,0.53,0.71,0.29,0.28,0.82
Bicycle theft,-0.036,0.53,1.0,0.59,0.59,0.49,0.33,0.68,0.6,0.61,0.69,0.51,0.64,0.32,0.39,0.73
Burglary,-0.096,0.61,0.59,1.0,0.71,0.49,0.36,0.72,0.62,0.63,0.64,0.69,0.74,0.29,0.32,0.8
Criminal damage and arson,-0.12,0.61,0.59,0.71,1.0,0.63,0.44,0.69,0.71,0.63,0.62,0.67,0.82,0.4,0.32,0.81
Drugs,-0.064,0.54,0.49,0.49,0.63,1.0,0.33,0.55,0.6,0.56,0.56,0.42,0.67,0.42,0.28,0.68
Other crime,-0.029,0.33,0.33,0.36,0.44,0.33,1.0,0.41,0.43,0.37,0.39,0.28,0.46,0.28,0.22,0.46
Other theft,-0.045,0.61,0.68,0.72,0.69,0.55,0.41,1.0,0.73,0.7,0.8,0.58,0.77,0.37,0.46,0.86
Public order,-0.09,0.61,0.6,0.62,0.71,0.6,0.43,0.73,1.0,0.67,0.68,0.51,0.81,0.4,0.42,0.81
Robbery,-0.084,0.58,0.61,0.63,0.63,0.56,0.37,0.7,0.67,1.0,0.73,0.55,0.72,0.38,0.36,0.78
